In [1]:
import numpy as np
import matplotlib.pyplot as plt  
import math   
import random   
import copy
import MEF

In [2]:
#vector con los valores incognitas 'r'
#vector con los valores de CC 's'
#U vector con los deplazamientos incognitas + datos.
#N= int(input('Ingrese el numero de nodos'))
N=4 #numero de nodos 'n'
k= 200 #constante de los resortes

In [3]:
MN = np.array([[0,0,0],[8,0,0],[16,0,0],[8,4,0]]) #parametro de entrada
MC= np.array([[0,3],[0,1],[1,3],[2,3],[1,2]]) #parametro de entrada
glxn= 2 #grados de libertad por nodo, parametro de entrada
E= np.array([210,210,210,210,210])*1E9
A = np.array([10,10,20,10,10])*1E-4

In [4]:
Nn = MN.shape[0] #cantidad de nodos
Ne, Nnxe= MC.shape #cantidad de elementos

In [5]:
Kglobal= np.zeros([glxn*Nn,glxn*Nn])

In [6]:
Kglobal= np.zeros([glxn*Nn,glxn*Nn])

In [7]:
def k_elem_barra(MC,MN,Ee,Ae,e):
    Nn = MN.shape[0] #cantidad de nodos
    Ne, Nnxe= MC.shape #cantidad de elementos
    nodo1= MC[e,0]
    nodo2= MC[e,1]
    cord_nodo1=np.array([MN[nodo2,0]-MN[nodo1,0],MN[nodo2,1]-MN[nodo1,1]])
    long= np.linalg.norm(cord_nodo1)
    phi= np.arctan2(cord_nodo1[1],cord_nodo1[0])
    ke= Ee*Ae/long
    c=np.cos(phi)
    s=np.sin(phi)
    K_elem= ke*np.array([[c**2,c*s,-c**2,-c*s],[c*s,s*s,-c*s,-s*s],[-c*c,-c*s,c*c,c*s],
               [-c*s,-s*s,c*s,s*s]])
    K_elem[np.abs(K_elem/K_elem.max()) < 1e-15] = 0

    return K_elem

In [8]:
k_elem= {}

In [9]:
archivo= 'Matrices_elementales.txt'
with open(archivo,'w') as f:
    f.write('Matrices Elementales\n ===============')
archivo1= 'Matriz_global.txt'
with open(archivo1,'w') as f:
    f.write('Matriz Global\n ===============')

In [10]:
for e in range(Ne):
    k_elem[e]= k_elem_barra(MC,MN,E[e],A[e],e)
    fe = np.abs(k_elem[e].max()) #factor de escala
    with open(archivo,'a') as f:
        f.write(f'\nelemento {e}, fe ={fe:4e}\n')
        f.write(f'{k_elem[e]/fe}\n')
    

    for i in range(Nnxe):
        rangoi= np.linspace(i*glxn,(i+1)*glxn-1,glxn, dtype = "int")
        rangoni= np.linspace(MC[e,i]*glxn,(MC[e,i]+1)*glxn-1,glxn, dtype = "int")
        for j in range(Nnxe):
            rangoj= np.linspace(j*glxn,(j+1)*glxn-1,glxn, dtype = "int")
            rangonj= np.linspace(MC[e,j]*glxn,(MC[e,j]+1)*glxn-1,glxn, dtype = "int")
            Kglobal[np.ix_(rangoni,rangonj)] += k_elem[e][np.ix_(rangoi,rangoj)]

In [11]:
fe = np.abs(Kglobal.max())
with open(archivo1,'a') as f:
    f.write(f'\nMatriz Global, fe ={fe:4e}\n')
    f.write(f'{Kglobal/fe}\n') 

In [12]:
s= np.array([0,1,5])
r = np.array([i for i in range(Nn*glxn) if i not in s ])
#s = np.array([0,1,5])
#r = np.arange(Nn*glxn)
#[r.remove(thiss) for thiss in s]

In [13]:
Fr = np.zeros_like(r)
Us = np.zeros_like(s)
Fr[r==3]= -20E3

In [14]:
U,F = MEF.solve1D(Kglobal,s,r,Us,Fr)

#  Calculo de tensiones

In [15]:
U

array([[ 0.        ],
       [ 0.        ],
       [ 0.0007619 ],
       [-0.00384387],
       [ 0.00152381],
       [ 0.        ],
       [ 0.0007619 ],
       [-0.0036534 ]])

In [16]:
U= U.reshape(Nn,2)

In [17]:
U

array([[ 0.        ,  0.        ],
       [ 0.0007619 , -0.00384387],
       [ 0.00152381,  0.        ],
       [ 0.0007619 , -0.0036534 ]])

In [18]:
aux= np.zeros([Nn,1])
despl= np.append(U, aux, axis=1)

In [19]:
MN_actualizada= despl + MN

In [28]:
long_final = []
long_inicial=[]
tensiones= []

In [29]:
for e in range(Ne):
    nodo1= MC[e,0]
    nodo2= MC[e,1]
    cord_nodo_inicial=np.array([MN[nodo2,0]-MN[nodo1,0],MN[nodo2,1]-MN[nodo1,1]])
    cord_nodo_final=np.array([MN_actualizada[nodo2,0]-MN_actualizada[nodo1,0],
                              MN_actualizada[nodo2,1]-MN_actualizada[nodo1,1]])
    long_final.append(np.linalg.norm(cord_nodo_final))
    long_inicial.append(np.linalg.norm(cord_nodo_inicial))
    tensiones.append((((long_final[e]-long_inicial[e])/long_inicial[e])*E[e]))

In [33]:
tensiones = np.array(tensiones)

In [34]:
tensiones/1E6 #estan en MPa

array([-22.34358817,  20.02423853,  10.        , -22.34358817,
        20.02423853])

In [35]:
long_inicial

[8.94427190999916, 8.0, 4.0, 8.94427190999916, 8.0]